<a href="https://colab.research.google.com/github/Yomex139/question-answering-app/blob/main/Questions_and__Answer_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 QUESTIONS AND ANSWER WEB APP PROJECT

<h1>QUESTIONS AND ANSWER WEB APP PROJECT</h1>

<h2>Import Libraries</h2>

In [ ]:
import numpy as np
from transformers import pipeline
import torch
import gradio as gr
from IPython.display import Markdown, display

<h2>Initiate Pipeline</h2>

In [ ]:
model_id = "distilbert/distilbert-base-cased-distilled-squad"
qa_pipeline = pipeline("question-answering", model=model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
#TEST THE qa_pipeline
Context = "The Amazon rainforest is the largest tropical rainforest in the world, spanning over 5.5 million square kilometers. It is home to an incredible diversity of wildlife, including jaguars, sloths, and hundreds of species of birds. The Amazon River, which runs through the forest, is the second-longest river in the world and plays a crucial role in the ecosystem. Deforestation and climate change pose significant threats to this rainforest, affecting both biodiversity and indigenous communities."
Question = "Which river flows through the Amazon rainforest?"
result = qa_pipeline(context=Context, question=Question)
print(f"Quesion: {Question}")
print(f"Answer: {result['answer']}")
print(f"Score: {result['score']*100:.2f}%")


Quesion: Which river flows through the Amazon rainforest?
Answer: Amazon River
Score: 50.72%


<h3>Define two Functions for the input</h3>

In [106]:
# Single Interactive Question function
def single_inetractive_question(context, question):
  '''
  accept a single question and context and return the answer
  '''
  result = qa_pipeline(context=context, question=question)
  answer = result['answer']
  confidence = f"{round(result['score']*100, 2)}%"
  highlighted = context.replace(answer, f"**{answer}**")
  return answer, confidence, highlighted

#MULTIPLE INPUT QUESTIONS FUNCTION
def multiple_interaction_questions(context, question):
  '''
  accept list of questions and context and return the answers
  '''
  answers = []

  questions = [q.strip() for q in question.strip().split("\n") if q.strip() != ""]
  for question in questions:
    result = qa_pipeline(context=context, question=question)
    answers.append(f"\nQuestion:\n {question}\nAnswer:\n {result['answer']}\nConfidence:\n {round(result['score']*100, 2)}%\n-------------------")
  return "".join(answers)

<h2>SETTING UP GRADIO INTERFACE</h2>

In [68]:
single_inetractive_question(context=Context, question=Question)

('Amazon River',
 '50.72%',
 'The Amazon rainforest is the largest tropical rainforest in the world, spanning over 5.5 million square kilometers. It is home to an incredible diversity of wildlife, including jaguars, sloths, and hundreds of species of birds. The **Amazon River**, which runs through the forest, is the second-longest river in the world and plays a crucial role in the ecosystem. Deforestation and climate change pose significant threats to this rainforest, affecting both biodiversity and indigenous communities.')

In [36]:
new_questions = '''What is the size of the Amazon rainforest?

Which animals live in the Amazon rainforest?

What role does the Amazon River play in the ecosystem?

What are the main threats to the Amazon rainforest?
'''

In [86]:
multiple_interaction_questions(context=Context, question=new_questions)

'Answer: 5.5 million square kilometers---- Confidence: 79.07---- Highlight: The Amazon rainforest is the largest tropical rainforest in the world, spanning over **5.5 million square kilometers**. It is home to an incredible diversity of wildlife, including jaguars, sloths, and hundreds of species of birds. The Amazon River, which runs through the forest, is the second-longest river in the world and plays a crucial role in the ecosystem. Deforestation and climate change pose significant threats to this rainforest, affecting both biodiversity and indigenous communities.\nAnswer: birds---- Confidence: 53.24---- Highlight: The Amazon rainforest is the largest tropical rainforest in the world, spanning over 5.5 million square kilometers. It is home to an incredible diversity of wildlife, including jaguars, sloths, and hundreds of species of **birds**. The Amazon River, which runs through the forest, is the second-longest river in the world and plays a crucial role in the ecosystem. Deforest

In [84]:
App_code = """with gr.Blocks() as app:
    gr.Markdown("# 📖 Question Answering Web App")
    gr.Markdown(
        "Paste a paragraph (context) and ask one or multiple questions.\n\n"
        "✅ Interactive questions → get answers one-by-one\n"
        "✅ Batch questions → enter multiple questions separated by newlines"
    )

    # Input: context
    context_input = gr.Textbox(label="Context", lines=6, placeholder="Paste your context paragraph here...")

    # ----- Interactive QA -----
    gr.Markdown("## 🤖 Interactive QA (One Question at a Time)")
    question_input = gr.Textbox(label="Question", placeholder="Ask a question about the context...")
    answer_output = gr.Textbox(label="Answer")
    confidence_output = gr.Textbox(label="Confidence Score")
    highlighted_output = gr.Textbox(label="Context with Highlighted Answer")

    interactive_btn = gr.Button("Get Answer")
    interactive_btn.click(
        single_inetractive_question,
        inputs=[context_input, question_input],
        outputs=[answer_output, confidence_output, highlighted_output]
    )

    # ----- Batch QA -----
    gr.Markdown("## 📄 Batch QA (Multiple Questions at Once)")
    batch_questions_input = gr.Textbox(label="Questions (one per line)", lines=5, placeholder="Enter multiple questions, each on a new line")
    batch_output = gr.Textbox(label="Answers")
    batch_btn = gr.Button("Get Batch Answers")
    batch_btn.click(multiple_interaction_questions, inputs=[context_input, batch_questions_input], outputs=batch_output)

# Launch app
app.launch()
"""
with open("app.py", "w") as f:
  f.write(App_code)

In [114]:
with gr.Blocks() as app:
    gr.Markdown("# 📖 Question Answering Web App")
    gr.Markdown(
        "Paste a paragraph (context) and ask one or multiple questions.\n\n"
        "✅ Interactive questions → get answers one-by-one\n"
        "✅ Batch questions → enter multiple questions separated by newlines"
    )

    # Input: context
    context_input = gr.Textbox(label="Context", lines=6, placeholder="Paste your context paragraph here...")

    # ----- Interactive QA -----
    gr.Markdown("## 🤖 Interactive QA (One Question at a Time)")
    question_input = gr.Textbox(label="Question", placeholder="Ask a question about the context...")
    answer_output = gr.Textbox(label="Answer")
    confidence_output = gr.Textbox(label="Confidence Score")
    highlighted_output = gr.Textbox(label="Context with Highlighted Answer")

    interactive_btn = gr.Button("Get Answer")
    interactive_btn.click(
        single_inetractive_question,
        inputs=[context_input, question_input],
        outputs=[answer_output, confidence_output, highlighted_output]
    )

    # ----- Batch QA -----
    gr.Markdown("## 📄 Batch QA (Multiple Questions at Once)")
    batch_questions_input = gr.Textbox(label="Questions (one per line)", lines=5, placeholder="Enter multiple questions, each on a new line")
    batch_output = gr.Textbox(label="Answers")
    batch_btn = gr.Button("Get Batch Answers")
    batch_btn.click(multiple_interaction_questions, inputs=[context_input, batch_questions_input], outputs=batch_output)

# Launch app
app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df2e304e094e793531.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
